In [1]:
!pip install -q llama-index
!pip install -q openai
!pip install -q transformers
!pip install -q accelerate
!pip install -q optimum[exporters]
!pip install -q InstructorEmbedding
!pip install -q sentence_transformers
!pip install -q pypdf
!pip install -q chromadb


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install -q llama-index chromadb --quiet
!pip install -q chromadb
!pip install -q sentence-transformers
!pip install -q pydantic==1.10.11


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
!pip install -q llama-index-embeddings-huggingface
!pip install -q llama-index-embeddings-openai


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
from dotenv import load_dotenv
import os

load_dotenv()  # This loads the variables from .env into the environment

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 8


In [40]:
# import
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from IPython.display import Markdown, display
import chromadb


In [41]:
# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [42]:
documents = SimpleDirectoryReader(
    "data"
).load_data()

# CHROMA-DB

In [24]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("ulstore")

### Set up ChromaVectorStore and load in data


In [25]:
# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


In [26]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [27]:
service_context = ServiceContext.from_defaults(embed_model=embed_model)

C:\Users\uzivatel\AppData\Local\Temp\ipykernel_8700\3553016389.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model)


In [28]:

index = VectorStoreIndex.from_documents(documents,
                                        storage_context=storage_context,
                                        service_context=service_context)

In [44]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("What is class 2 circuit and what are its propperties?")

display(Markdown(f"{response}"))

A Class 2 circuit is a type of power supply that meets specific safety requirements. It is characterized by having a maximum output voltage of 42.4 V peak AC (30 V RMS) or 60 V DC. Class 2 circuits are designed to limit the risk of electric shock and fire hazards. Additionally, they have no direct electrical connection between the input and output, typically achieved through the use of transformers, optical isolators, or other suitable means.

            What cable do I use to hang a 1.5kg heavy luminaire on?
            What is the minimal gauge of live wires ?
            What flamability fequirements do plastic enclosure have to meet ?
            What is class 2 circuit and what are its propperties?

### How to Persist: Saving to Disk

In [33]:
db = chromadb.PersistentClient(path="./chroma_db")


chroma_collection = db.get_or_create_collection("512_store")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

service_context = ServiceContext.from_defaults(embed_model=embed_model,
                                               chunk_size=512,
                                               chunk_overlap=128)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

C:\Users\uzivatel\AppData\Local\Temp\ipykernel_8700\1637600030.py:10: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model,


### Load from Disk

In [34]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")

chroma_collection = db2.get_or_create_collection("512_store")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [38]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("What is the minimal thickness of the matalic enclosure sheet?")

display(Markdown(f"{response}"))

The minimal thickness of the metallic enclosure sheet is 0.66 mm (0.026 in) for unreinforced steel and extruded aluminum at the opening for conduit connection.

# PINECONE

In [ ]:
!pip install -q pinecone-client

In [ ]:
import pinecone

In [ ]:
os.environ['PINECONE_API_KEY'] = "PINECONE_API_KEY"
os.environ['PINECONE_ENVIRONMENT'] = 'gcp-starter'


In [ ]:
# initialize connection to pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENVIRONMENT']
)

In [ ]:
# create the index if it does not exist already
index_name = 'llamaindex'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=768,
        metric='cosine'
    )

In [ ]:
pinecone_index = pinecone.Index("llamaindex")
pinecone.describe_index("llamaindex")

IndexDescription(name='llamaindex', metric='cosine', replicas=1, dimension=768.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [ ]:
from llama_index.vector_stores import PineconeVectorStore

In [ ]:
# set add_sparse_vector=True to compute sparse vectors during upsert
from llama_index.storage.storage_context import StorageContext

In [ ]:
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index,
    # add_sparse_vector=True,
)


In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(embed_model=embed_model)


In [ ]:
index = VectorStoreIndex.from_documents(documents,
                                        storage_context=storage_context,
                                        service_context=service_context
                                        )

Upserted vectors:   0%|          | 0/55 [00:00<?, ?it/s]

In [ ]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("What is Big-Bench Hard test?")

display(Markdown(f"{response}"))

Big-Bench Hard is a suite of 23 challenging tasks that were introduced to measure the capabilities and limitations of large language models. These tasks are specifically designed to be difficult and have been selected based on the fact that prior language models did not outperform the average human-rater on them. The evaluation of Big-Bench Hard tasks is done using standard zero-shot prompting, without the use of any labeled examples.